In [27]:
from hmmlearn import hmm
import glob
import os
from numpy import load
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, f1_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn import metrics
from tqdm import tqdm
import operator
from scipy.special import softmax
import importlib
import hmm_helper
from sklearn.model_selection import LeaveOneOut,LeaveOneGroupOut
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from scipy import interp
import pylab as pl
from matplotlib.pyplot import figure
from sklearn.metrics import confusion_matrix
import itertools
import statistics
from sklearn.metrics.scorer import make_scorer
import series_utils
from sklearn.pipeline import Pipeline


[autoreload of series_utils failed: Traceback (most recent call last):
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/jarvis/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 266, in 

In [28]:
%load_ext autoreload
%autoreload 2

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
importlib.reload (hmm_helper)

<module 'hmm_helper' from '/home/jarvis/Documentos/catalina/hmm/hmm_helper.py'>

## Define paths and ref files


In [30]:
dirpath = '/home/jarvis/Documentos/catalina/hmm/TIME_SERIES/'
subjects = [os.path.basename(x) for x in glob.glob(dirpath + '*sub*')]
scans = ['_scan_restpre', '_scan_restpost']
stimuli = ['ses-occ','ses-ctr', 'ses-fro' ]

parcel="_mask_Schaefer2018_400_17N_MNI152_2mm_mask_file_..input..data..Schaefer2018_400_17N_MNI152_2mm.nii.gz"
np.random.seed(42)

In [31]:
schafer_file = '/home/jarvis/Documentos/catalina/hmm/tms_hmm/Schaefer2018_400_17N_Centroid_RAS.csv'
schafer = pd.read_csv(schafer_file, header=None, skiprows=[0,1])
schafer['network'] = schafer[1].str.split('_').str.get(2)
schafer['network_id'] = schafer.network.astype("category").cat.codes
schafer_image = '/home/jarvis/Documentos/catalina/hmm/tms_hmm/Schaefer2018_400_17N_MNI152_2mm.nii'

## GRIDSEARCH

In [32]:
def f1_func(y_true, y_pred, **kwargs):
    print('y_true ',y_true )
    print('y_ped ',y_pred )
    return f1_score(y_true, y_pred.reshape(-1,1))
scorer_f1 = make_scorer(f1_func, greater_is_better=True)

In [33]:
def accuracy_func(y_true, y_pred, **kwargs):
    print('y_true ',y_true )
    print('y_ped ',y_pred )
    return accuracy_score(y_true, y_pred.reshape(-1,1))
scorer_accuracy = make_scorer(accuracy_func, greater_is_better=True)

## Load Data
#### Based on get_all_data from series_utils.py 

## OCC

In [34]:
occ_pre, subjects_list_occ_pre = series_utils.get_all_data(['_scan_restpre'], ['ses-occ'], subjects, dirpath, parcel,schafer)
occ_post, subjects_list_occ_post =  series_utils.get_all_data(['_scan_restpost'], ['ses-occ'], subjects, dirpath, parcel,schafer)

In [35]:
all_occ = occ_pre + occ_post
subjects_occ = subjects_list_occ_pre + subjects_list_occ_post
y_occ = [0]*len(subjects_list_occ_pre) + [1]*len(subjects_list_occ_post)

## Transform dimensions to get shape #subject x #timepoints x #variables


In [36]:
X_occ = np.swapaxes(np.swapaxes(np.dstack(all_occ), 0,2),1,2)
y_occ = np.array(y_occ)
print('X shape:', X_occ.shape)
print('y shape: ', y_occ.shape)

X shape: (46, 600, 15)
y shape:  (46,)


In [37]:
cat_subjects = pd.get_dummies(subjects_occ)

subjects_id_occ = cat_subjects.values.argmax(1)

class_names = ['Pre', 'Post']
print(subjects_id_occ)

[ 8  6 20 17  7 22 14  0 21  5  9  2 16 19  1  4 13 18  3 10 12 15 11  8
  6 20 17  7 22 14  0 21  5  9  2 16 19  1  4 13 18  3 10 12 15 11]


In [38]:
gKFold = LeaveOneGroupOut
itera = 500
scaler_3d = series_utils.StandardScaler3D()

base_model = hmm.GMMHMM(covariance_type="diag",init_params="cm", params="cm")
model_hmm = hmm_helper.HMM_classifier(base_model)


pipe = Pipeline(steps=[("scaler", scaler_3d), ("model", model_hmm)])

parameters = { 'model__hmm_model': [base_model],
                'model__n_components':[*range(3, 16)],
                'model__n_mix': [*range(1, 10)]
}


grid_search = GridSearchCV(pipe, parameters, cv = LeaveOneGroupOut(), scoring=scorer_accuracy, verbose = 1,n_jobs=-1)

grid_search.fit(X = X_occ, y = y_occ, groups = subjects_id_occ)

Fitting 23 folds for each of 117 candidates, totalling 2691 fits


GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler3D()),
                                       ('model',
                                        HMM_classifier(hmm_model=GMMHMM(init_params='cm',
                                                                        params='cm')))]),
             n_jobs=-1,
             param_grid={'model__hmm_model': [GMMHMM(init_params='cm',
                                                     params='cm')],
                         'model__n_components': [3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                 12, 13, 14, 15],
                         'model__n_mix': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             scoring=make_scorer(accuracy_func), verbose=1)

In [39]:
grid_search.best_score_

0.5869565217391305

In [40]:
print('Best Score: %s' % grid_search.best_score_)
print('Best Hyperparameters: %s' % grid_search.best_params_)

Best Score: 0.5869565217391305
Best Hyperparameters: {'model__hmm_model': GMMHMM(init_params='cm', params='cm'), 'model__n_components': 6, 'model__n_mix': 1}


## CRONTROL

In [41]:
pre_dataset_ctr, subjects_list_pre_ctr = series_utils.get_all_data(['_scan_restpre'], ['ses-ctr'], subjects, dirpath, parcel,schafer)
ctr_post, subjects_list_ctr_post =  series_utils.get_all_data(['_scan_restpost'], ['ses-ctr'], subjects, dirpath, parcel,schafer)

In [42]:
all_ctr = pre_dataset_ctr + ctr_post
subjects_ctr = subjects_list_pre_ctr + subjects_list_ctr_post
y_ctr = [0]*len(subjects_list_pre_ctr) + [1]*len(subjects_list_ctr_post)

## Transform dimensions to get shape #subject x #timepoints x #variables

In [43]:
X_ctr = np.swapaxes(np.swapaxes(np.dstack(all_ctr), 0,2),1,2)
y_ctr = np.array(y_ctr)
print('X shape:', X_ctr.shape)
print('y shape: ', y_ctr.shape)

X shape: (46, 600, 15)
y shape:  (46,)


In [44]:
cat_subjects_ctr = pd.get_dummies(subjects_ctr)

subjects_id_ctr = cat_subjects_ctr.values.argmax(1)

class_names = ['Pre', 'Post']
subjects_id_ctr

array([ 8,  6, 20, 17,  7, 22, 14,  0, 21,  5,  9,  2, 16, 19,  1,  4, 13,
       18,  3, 10, 12, 15, 11,  8,  6, 20, 17,  7, 22, 14,  0, 21,  5,  9,
        2, 16, 19,  1,  4, 13, 18,  3, 10, 12, 15, 11])

In [45]:
gKFold = LeaveOneGroupOut
itera = 500
scaler_3d = series_utils.StandardScaler3D()

base_model = hmm.GMMHMM(covariance_type="diag",init_params="cm", params="cm")
model_hmm = hmm_helper.HMM_classifier(base_model)

pipe = Pipeline(steps=[("scaler", scaler_3d), ("model", model_hmm)])

parameters = { 'model__hmm_model': [base_model],
                'model__n_components':[*range(3, 16)],
                'model__n_mix': [*range(1, 10)]
}


grid_search = GridSearchCV(pipe, parameters, cv = LeaveOneGroupOut(), scoring=scorer_accuracy, verbose = 1,n_jobs=-1)

grid_search.fit(X = X_ctr, y = y_ctr, groups = subjects_id_ctr)

Fitting 23 folds for each of 117 candidates, totalling 2691 fits


GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler3D()),
                                       ('model',
                                        HMM_classifier(hmm_model=GMMHMM(init_params='cm',
                                                                        params='cm')))]),
             n_jobs=-1,
             param_grid={'model__hmm_model': [GMMHMM(init_params='cm',
                                                     params='cm')],
                         'model__n_components': [3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                 12, 13, 14, 15],
                         'model__n_mix': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             scoring=make_scorer(accuracy_func), verbose=1)

In [46]:
print('Best Score: %s' % grid_search.best_score_)
print('Best Hyperparameters: %s' % grid_search.best_params_)

Best Score: 0.4782608695652174
Best Hyperparameters: {'model__hmm_model': GMMHMM(init_params='cm', params='cm'), 'model__n_components': 3, 'model__n_mix': 2}


## FRONTAL

In [47]:
pre_dataset_fro, subjects_list_pre_fro = series_utils.get_all_data(['_scan_restpre'], ['ses-fro'], subjects, dirpath, parcel,schafer)
fro_post, subjects_list_fro_post =  series_utils.get_all_data(['_scan_restpost'], ['ses-fro'], subjects, dirpath, parcel,schafer)

In [48]:
all_fro = pre_dataset_fro + fro_post
subjects_fro = subjects_list_pre_fro + subjects_list_fro_post
y_fro = [0]*len(subjects_list_pre_fro) + [1]*len(subjects_list_fro_post)

## Transform dimensions to get shape #subject x #timepoints x #variables

In [49]:
X_fro = np.swapaxes(np.swapaxes(np.dstack(all_fro), 0,2),1,2)
y_fro = np.array(y_fro)
print('X shape:', X_fro.shape)
print('y shape: ', y_fro.shape)

X shape: (46, 600, 15)
y shape:  (46,)


In [50]:
cat_subjects_fro = pd.get_dummies(subjects_fro)

subjects_id_fro = cat_subjects_fro.values.argmax(1)

class_names = ['Pre', 'Post']
subjects_id_fro

array([ 8,  6, 20, 17,  7, 22, 14,  0, 21,  5,  9,  2, 16, 19,  1,  4, 13,
       18,  3, 10, 12, 15, 11,  8,  6, 20, 17,  7, 22, 14,  0, 21,  5,  9,
        2, 16, 19,  1,  4, 13, 18,  3, 10, 12, 15, 11])

In [51]:
gKFold = LeaveOneGroupOut
itera = 500
scaler_3d = series_utils.StandardScaler3D()

base_model = hmm.GMMHMM(covariance_type="diag",init_params="cm", params="cm")
model_hmm = hmm_helper.HMM_classifier(base_model)

pipe = Pipeline(steps=[("scaler", scaler_3d), ("model", model_hmm)])

parameters = { 'model__hmm_model': [base_model],
                'model__n_components':[*range(3, 16)],
                'model__n_mix': [*range(1, 10)]
}


grid_search = GridSearchCV(pipe, parameters, cv = LeaveOneGroupOut(), scoring=scorer_accuracy, verbose = 1,n_jobs=-1)

grid_search.fit(X = X_fro, y = y_fro, groups = subjects_id_ctr)

Fitting 23 folds for each of 117 candidates, totalling 2691 fits
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_p

y_true  [0 1]y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

...

y_true  [0 1]y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

y_true  [0 1]y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [0 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [0 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 0]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y_ped  [1 1]
y_true  [0 1]
y

GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler3D()),
                                       ('model',
                                        HMM_classifier(hmm_model=GMMHMM(init_params='cm',
                                                                        params='cm')))]),
             n_jobs=-1,
             param_grid={'model__hmm_model': [GMMHMM(init_params='cm',
                                                     params='cm')],
                         'model__n_components': [3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                 12, 13, 14, 15],
                         'model__n_mix': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             scoring=make_scorer(accuracy_func), verbose=1)

In [52]:
print('Best Score: %s' % grid_search.best_score_)
print('Best Hyperparameters: %s' % grid_search.best_params_)

Best Score: 0.5434782608695652
Best Hyperparameters: {'model__hmm_model': GMMHMM(init_params='cm', params='cm'), 'model__n_components': 3, 'model__n_mix': 2}
